# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


In [3]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row, SparkSession
from pyspark.sql import functions as F

# 创建SparkSession对象
spark = SparkSession.builder \
    .appName("Lab") \
    .getOrCreate()

## 创建SparkSession对象，然后调用里面的SparkContext方法读取text文件（如果想使用rdd转换为dataframe，
## 需要先用rdd读入文件处理，再使用spark.createDataFrame转换为DataFrame。）

In [8]:
## 这里使用spark.sparkContext.textFile读入text文件并载入为rdd
## 记得之后更换地址和文件名

temperature_file = spark.sparkContext.textFile("s3://bigdatalabs7051/data/temperature-readings-small/temperature-readings-small.csv")
lines = temperature_file.map(lambda line: line.split(";"))

# (key, value) = (year,temperature)
tempReadings = lines.map(lambda p: Row(station=p[0],
                                       date=p[1],
                                       year=p[1].split("-")[0],
                                       time=p[2],
                                       value=float(p[3]),
                                       quality=p[4]))

schemaTempReadings = spark.createDataFrame(tempReadings)

## 如果不用sql风格处理，不需要用schemaTempReadings.registerTempTable创建视图

schemaTempReadings.show()

+----------+-------+-------+--------+-----+----+
|      date|quality|station|    time|value|year|
+----------+-------+-------+--------+-----+----+
|2013-11-01|      G| 102170|06:00:00|  6.8|2013|
|2013-11-01|      G| 102170|18:00:00|  3.8|2013|
|2013-11-02|      G| 102170|06:00:00|  5.8|2013|
|2013-11-02|      G| 102170|18:00:00| -1.1|2013|
|2013-11-03|      G| 102170|06:00:00| -0.2|2013|
|2013-11-03|      G| 102170|18:00:00|  5.6|2013|
|2013-11-04|      G| 102170|06:00:00|  6.5|2013|
|2013-11-04|      G| 102170|18:00:00|  5.1|2013|
|2013-11-05|      G| 102170|06:00:00|  4.2|2013|
|2013-11-05|      G| 102170|18:00:00|  3.2|2013|
|2013-11-06|      G| 102170|06:00:00|  1.7|2013|
|2013-11-06|      G| 102170|18:00:00|  0.9|2013|
|2013-11-07|      G| 102170|06:00:00| -0.1|2013|
|2013-11-07|      G| 102170|18:00:00|  0.1|2013|
|2013-11-08|      G| 102170|06:00:00| -1.2|2013|
|2013-11-08|      G| 102170|18:00:00|  5.3|2013|
|2013-11-09|      G| 102170|06:00:00|  5.6|2013|
|2013-11-09|      G|

In [9]:
TempMinByYearStation = schemaTempReadings.where((schemaTempReadings["year"] >= 1950) & (schemaTempReadings["year"] <= 2014)). \
    groupBy('year', 'station'). \
    agg(F.min('value').alias('annualMin')). \
    select('year', 'station', 'annualMin')
    # orderBy(['annualMin'], ascending=[False])
    
TempMinByYearStation.show()

+----+-------+---------+
|year|station|annualMin|
+----+-------+---------+
|2014| 102170|    -24.3|
|2013| 102170|    -13.3|
+----+-------+---------+


In [12]:
# TempMinByYear = schemaTempReadings.groupBy('year'). \
#     agg(F.min('annualMin').alias('annualMin'))

## schemaTempReadings中没有annualMin列，我猜你是想用TempMinByYearStation表

TempMinByYear = TempMinByYearStation.groupBy('year'). \
    agg(F.min('annualMin').alias('annualMin'))
TempMinByYear.show()

+----+---------+
|year|annualMin|
+----+---------+
|2014|    -24.3|
|2013|    -13.3|
+----+---------+


In [21]:
# TempMinByYearWithStation = TempMinByYearStation.join(TempMinByYear, on=['year', 'annualMin'])\
#     .groupBy('year', 'annualMin')\
#     .select('year', 'station', 'annualMin')\
#     .orderBy(['annualMin'], ascending=[False])

TempMinByYearWithStation = TempMinByYearStation.join(TempMinByYear, on=['year', 'annualMin'])\
    .select('year', 'station', 'annualMin')\
    .orderBy(['annualMin'], ascending=[False])

TempMinByYearWithStation.show()

## 记得遵循语句顺序，先select再groupBy,然后聚合
## 这里为什么要用groupBy？

+----+-------+---------+
|year|station|annualMin|
+----+-------+---------+
|2013| 102170|    -13.3|
|2014| 102170|    -24.3|
+----+-------+---------+


In [27]:
# ------max
TempMaxByYearStation = schemaTempReadings.where((schemaTempReadings["year"] >= 1950) & (schemaTempReadings["year"] <= 2014)). \
    groupBy('year', 'station'). \
    agg(F.max('value').alias('annualMax')). \
    select('year', 'station', 'annualMax')
    # orderBy(['annualMax'], ascending=[False])

TempMaxByYear = TempMaxByYearStation.groupBy('year'). \
    agg(F.max('annualMax').alias('annualMax'))

TempMaxByYearWithStation = TempMaxByYearStation.join(TempMaxByYear, on=['year', 'annualMax'])\
    .select('year', 'station', 'annualMax')\
    .orderBy(['annualMax'], ascending=[False])

TempMaxByYearWithStation.show()

+----+-------+---------+
|year|station|annualMax|
+----+-------+---------+
|2014| 102170|     29.1|
|2013| 102170|     10.2|
+----+-------+---------+


In [28]:
# Following code will save the result into /user/ACCOUNT_NAME/BDA/output folder

## 你可以存储为csv文件，存储表格比rdd更方便

TempMaxByYearWithStation.write.mode("overwrite").format("csv").save("s3://bigdatalabs7051/lab2/lab2012")
TempMinByYearWithStation.write.mode("overwrite").format("csv").save("s3://bigdatalabs7051/lab2/lab2011")